In [1]:
import pandas as pd
import geopandas as gpd
import sqlalchemy as sa
import pyodbc
import tkinter as tk
from sqlalchemy import create_engine
from api_config import geocode_postcode, fetch_isochrone, PostcodeNotFound
from geo_config import isochrone_to_gdf, teams_to_gdf, filter_teams_by_minutes,list_business_units, apply_team_filters

In [2]:
# Define custom colour palette

humannature = {'GC Dark Green': '#294238',
               'GC Light Green': '#b2d235',
               'GC Mid Green': '#50b748',
               'GC Orange': '#f57821',
               'GC Light Grey': '#e6ebe3'
               }

In [3]:
# SQL server details 
server = 'dblistener1'
database = 'information_centre'
driver = 'ODBC Driver 17 for SQL Server' # Create SQLAlchemy engine and install necessary drivers
connection_string = f'mssql+pyodbc://{server}/{database}?driver={driver}'
engine = create_engine(connection_string)

# SQL query to select dataset from SSMS tables and save into allvisits dataframe
query = """
SELECT
    a.intContractorID,
    a.strName AS Contractor,
    e.BusinessUnitID,
    e.Name AS BusinessUnit,
    UPPER(a.strPostcode) AS Postcode,
    g.Latitude,
    g.Longitude,
    a.InternalContractor
FROM
    tblContractor a
    LEFT JOIN Contractor_Business_Unit d
        ON a.intContractorID = d.ContractorID
    LEFT JOIN Business_Unit e
        ON d.BusinessUnitID = e.BusinessUnitID
    LEFT JOIN Business_Unit_Master_Status f
        ON d.StatusID = f.StatusID
    LEFT JOIN dbs_PostCode.dbo.tblPostcodes_New g
        ON REPLACE(a.strPostcode, ' ', '') = g.PostcodeNoSpaces COLLATE Latin1_General_CI_AS
WHERE
    ISNULL(a.bDisabled, 0) = 0
    AND f.StatusID IN (60, 70, 80)
    AND ISNULL(a.IsTest, 0) = 0
    AND e.BusinessUnitID != 37;
"""
with engine.connect() as connection:
    fieldteams = pd.read_sql(query, connection)
    connection.close()

In [ ]:
try:
    site_lon, site_lat = geocode_postcode("CM12 0EQ")
except PostcodeNotFound as e:
    # In the GUI: show a messagebox with str(e) and return early
    print(f"Postcode error: {e}")
else:
    # One API call to Mapbox Isochrone API
    iso_geojson = fetch_isochrone(site_lon, site_lat)

    # Filter locally using GeoPandas rather than multiple API calls
    iso_gdf   = isochrone_to_gdf(iso_geojson)
    teams_gdf = teams_to_gdf(fieldteams)

    # Filter teams within the 60-minute isochrone to reduce multiple API calls later
    filtered_teams = filter_teams_by_minutes(teams_gdf, iso_gdf, minutes=60)

# Business Unit options for dropdown
bu_options = list_business_units(filtered_teams)

# Example selections
chosen_bu = "(Any)"         # or something from bu_options, e.g., "Arboriculture"
chosen_internal_flag = None # 1 = Directly Employed, 0 = Contractor, None = both

# Apply filters without making any OSRM calls yet
teams_ready_for_routing = apply_team_filters(
    filtered_teams,
    business_unit=chosen_bu,
    internal_flag=chosen_internal_flag
)


Teams after BU/internal filters: 160


,intContractorID,Contractor,BusinessUnitID,BusinessUnit,Postcode,Latitude,Longitude,InternalContractor,geometry
0,4713,OPERATIONS London and East Region,7,Grounds Maintenance,EC3N 1LS,51.513279,-0.075934,1,POINT (-0.07593 51.51328)
1,7550,Manor Oaks Ltd,7,Grounds Maintenance,SS8 9QP,51.530073,0.573521,0,POINT (0.57352 51.53007)
2,7550,Manor Oaks Ltd,33,GM & Pest Control,SS8 9QP,51.530073,0.573521,0,POINT (0.57352 51.53007)
3,3988,Valiant Arborist Ltd,35,Utility Arboriculture - New,SS92AD,51.548736,0.651186,0,POINT (0.65119 51.54874)
4,3988,Valiant Arborist Ltd,3,Arboriculture Utility,SS92AD,51.548736,0.651186,0,POINT (0.65119 51.54874)
